@author: Boda Ye
This python file used to label conn log file by using honeypot logfile.

        def ip_list:        create honeypot ip_time dictionary
                            like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}



    class label: utilize the ip_time dicitonary to label the conn log file



        def subset_conn:    This function is used to subset the conn log file, pick out rows with 'malicious ip'
                            (ip caught in honeypot)



        def preprocess_ip:  This function is used to convert the ip_time dicitonary to ip_start_end dictionary 
                            The unit for window is sec
                            
                            example:
                            ip1: {10,15,100} window=100, ts_threshold=13
                            => ip1:{10,100}
                            => ip1:{10-110, 100-200} 
                            => ip1:{10-200}
                           
                            

        def mal_uid:        return a list of malicious uid
    

In [1]:
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime

In [2]:

#create honeypot ip_time dictionary
#like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}
#input honeypotfile should be csv
def ip_list(honeypot):
    start_time = time.time()
    
    ip_time={}
    f=open(honeypot,'r')
    line=f.readline().strip()
    
    while line:
        try:
            temp=line.split(',')
            ts=int(time.mktime(datetime.strptime(temp[0]+temp[1][:-7], '%Y-%m-%d%H:%M:%S').timetuple()))
            sip=temp[5]
            
            if sip=='-':
                line=f.readline().strip()
                continue
            
            #print(str(ts)+' '+str(sip))
            if sip not in ip_time.keys():
                ip_time[sip]={ts}
            else:
                ip_time[sip].add(ts)
                
            
        except:
            pass
        
        line=f.readline().strip()
        
    
    
    f.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    #return a dictionary
    return ip_time

In [3]:
#create honeypot ip_time dictionary
#like, { ip1: {time1,time2}, ip2:{time5, time 6, time7}}
inputFile='/scratch/by8jj/merged_honeypot.csv'
ip_time=ip_list(inputFile)

--- 141.16733264923096 seconds ---


In [4]:
len(ip_time)

3288

In [5]:
class label(object):
    
    
    def __init__(self,conn):
        self.conn=conn

    
    #a mini function to check whether there's overlap between 2 duration
    def overlap(self,a,b):
        if a[1]<b[0]:return True
        return False
    
    
    #This function is used to convert the ip_time dicitonary to ip_start_end dictionary 
    #The unit for window is sec
    def preprocess_ip(self,ip_time,ts_threshold=13,window=43200):
        
        start_time=time.time()
        #To simplify problem, I assume logs with same ip in 13 secs are from exactly the same malicious hacker
        #choose 13 here is because of mean duration for connection attempts is 13 secs
        
        out={}
        for key,value in ip_time.items():

            #combine connection happened within 13 secs to reduce size of time_stamp file
            ts=[int(x) for x in value]
            temp=[ts[0]]
            for j in range(len(ts)-1):
                if ts[j+1]-ts[j]>ts_threshold: temp.append(ts[j+1])           


            #create the start_end for each item in temp
            temp=[(x,x+window) for x in temp]
            #merge start_end
            merged_start_end=[temp[0]]
            for j in range(1,len(temp)):
                current_start_end=merged_start_end.pop()
                if self.overlap(current_start_end,temp[j]):
                    merged_start_end.append((current_start_end[0],temp[j][1]))
                else:
                    merged_start_end.append(current_start_end)
                    merged_start_end.append(temp[j])
            out[key]=merged_start_end
    
        #return a dictionary 
        print('preprocess ip:--------%s seconds ----------'%(time.time()-start_time))
        return out
        
        
    #label the file
    def label(self,ip_time,outfile='labeled.log',l_ts_threshold=13,l_window=43200):
        
        #intitialize some variable here
        start_time=time.time()
        mal_ips=[]
        mal_count=0
        
        
        #run preprocess function
        preprocess_ip_time=self.preprocess_ip(ip_time,ts_threshold=l_ts_threshold,window=l_window)
        
        
        #initialize the output file
        out=open(outfile,'w')
        
        
        #read in the file
        f = open(self.conn, 'r')
        
        line=f.readline().strip()
        
        counter=1
        
        while line:
            
            if '#' in line:
                line=f.readline().strip()
                counter+=1
                continue
                
            lsplit = line.split('\t')     
            ts, uid, ip = lsplit[0][:10],lsplit[1], lsplit[2]

            #check if ip is caught in honeypot file
            if ip not in preprocess_ip_time.keys():
                out.write(line+'\t'+'0'+'\n')
                line=f.readline().strip()
                counter+=1
                continue

            #check if time is within the range on the preprocessed ip_time file
            for j in preprocess_ip_time[ip]:
                if int(ts)>=int(j[0]) and int(ts)<=int(j[1]):
                    mal_count+=1
                    mal_ips.append(ip)
                    out.write(line+'\t'+'1'+'\n')
                    line=f.readline().strip()
                    break
            else:
                out.write(line+'\t'+'0'+'\n')
                line=f.readline().strip()
                

            counter+=1
        
        f.close()
        out.close()
        print('size of original file is:%s, size of malicious ip in conn log file is:%s '%(counter, mal_count))
        print('label:--------%s seconds -------'%(time.time()-start_time))
        return mal_ips

In [6]:
conn_orig=[
      '/scratch/by8jj/august_conn_logs/2018-08-10/conn_08_10.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-11/conn_08_11.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-12/conn_08_12.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-13/conn_08_13.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-14/conn_08_14.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-15/conn_08_15.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-16/conn_08_16.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-17/conn_08_17.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-18/conn_08_18.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-19/conn_08_19.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-20/conn_08_20.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-21/conn_08_21.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-22/conn_08_22.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-23/conn_08_23.log',\
      '/scratch/by8jj/august_conn_logs/2018-08-24/conn_08_24.log',\
     ]
      

In [7]:
conn_labelled=['labelled_08_10.log','labelled_08_11.log','labelled_08_12.log',\
               'labelled_08_13.log','labelled_08_14.log','labelled_08_15.log',\
               'labelled_08_16.log','labelled_08_17.log','labelled_08_18.log',\
               'labelled_08_19.log','labelled_08_20.log','labelled_08_21.log',\
               'labelled_08_22.log','labelled_08_23.log','labelled_08_24.log',\
              ]

In [ ]:

for i,j in zip(conn_orig,conn_labelled):
    print(j)
    try:
        p=label(i)
        temp=p.label(ip_time,outfile=j)
        df=pd.DataFrame({'ip':temp})
        df.to_csv('malicious_ip_'+j.split('.')[0]+'.csv',index=False)
    except:
        print('ERROR!!')
    
    

labelled_08_10.log
preprocess ip:--------0.07497143745422363 seconds ----------
size of original file is:250789993, size of malicious ip in conn log file is:1196439 
label:--------497.7291190624237 seconds -------
labelled_08_11.log
preprocess ip:--------0.0668635368347168 seconds ----------


In [ ]:
#implement sensitivity analysis to decide the best time window
def mali_sensitivity(start=10800,end=21600,step=3600):
    res=[]
    for i in range(start,end,step):      
        temp=p.label(ip_time,window=i)
        res.append(temp)
        print(temp)
        print("--- %s seconds ---" %(time.time() - start_time))
    return res

In [ ]:
#tiny function to view the log file
def view(file,row=100):
    f=open(file,'r')

    for i in range(row):
        line=f.readline().strip()
        print(line)
    

In [ ]:
#test(conn)